In [32]:
import pandas as pd

In [3]:
scs = pd.read_csv('../raw_data/SCS alerts Nov.csv')

In [4]:
def add_code(data):
    """
    
    """
    scs = data.copy()
    scs['code'] = scs['Alert'].str.extract('(^[A-Z]{3}[0-9]{3}|[A-Z][0-9]{4}[A-Z]{3}[0-9]{3}|[A-Z]{3} [A-Z][0-9]{2})')
    scs['code'] = scs['Alert'].str.extract('([A-Z][0-9]{4}[A-zZ]{3}[0-9]{3})')
    
    scs.loc[scs['code'].isna(), 'code'] = scs.loc[scs['code'].isna(), 'PLC']
    
    #scs['Pick Station'] = scs['Alert'].str.extract('(PTT[0-9]{3})')[0]
    return scs

In [25]:
def add_tote_colour(scs_code, asset_lu):
    """
    
    """
    df_totes = pd.merge(scs_code, asset_lu.drop('Number', axis=1), how='left', on='code')
    df_totes.loc[df_totes['PLC'].isin(['C17', 'C16', 'C15', 'C23']), 'Tote Colour'] = 'Blue'
    df_totes['Pick Station'] = df_totes['Alert'].str.contains('PTT[0-9]{3}')
    df_totes.loc[df_totes['Pick Station'], 'Tote Colour'] = 'Both'
    df_totes['PLC_number'] = df_totes['PLC'].str.extract('((?<=C)[0-9]{2})').fillna(0).astype('int')
    df_totes.loc[df_totes['PLC_number'] > 34, 'Tote Colour'] = 'Blue'
    
    # Unmapped
    unmapped = df_totes[df_totes['Tote Colour'].isna()]['code'].value_counts().reset_index().copy()
    unmapped = unmapped.rename(columns={'index' : 'Asset', 'code' : 'Occurrence'})
    # Map unknown to Both
    df_totes.loc[df_totes['Tote Colour'].isna(), 'Tote Colour'] = 'Both'
    return df_totes, unmapped

In [26]:
asset_lu = pd.read_excel('../raw_data/asset_tote_lookup.xlsx', sheet_name='new_look_up')

In [27]:
scs_code = add_code(scs)

In [28]:
mapped, unmapped = add_tote_colour(scs_code, asset_lu)

In [30]:
unmapped.to_excel('../raw_data/unmapped_assets.xlsx', index=False)